<a href="https://colab.research.google.com/github/frontier1020/Korbert_Start/blob/master/korbert_start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import keras

In [0]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical

In [197]:
import os
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [199]:
!pip install keras-bert

In [0]:
from keras_bert import get_pretrained, PretrainedList, get_checkpoint_paths, load_trained_model_from_checkpoint, extract_embeddings

In [203]:
model_path = get_pretrained(PretrainedList.multi_cased_base)
paths = get_checkpoint_paths(model_path)
print(paths.config, paths.checkpoint, paths.vocab)

/root/.keras/datasets/multi_cased_L-12_H-768_A-12/bert_config.json /root/.keras/datasets/multi_cased_L-12_H-768_A-12/bert_model.ckpt /root/.keras/datasets/multi_cased_L-12_H-768_A-12/vocab.txt


In [0]:
dataset = pd.read_csv('/content/gdrive/My Drive/팀프로젝트(공유용)/03_01_05_Korbert_Start/review.csv')
input_data = dataset['리뷰'].tolist()
output_data = dataset['점수'].tolist()
SEQ_LEN = len(max(input_data, key=len))

In [0]:
def convert_data(input_data, output_data):
    global tokenizer
    indices, targets = [], []
    for i in tqdm(range(len(input_data))):
        ids, segments = tokenizer.encode(input_data[i], max_len=SEQ_LEN)
        indices.append(ids)
        output_data_ohe = to_categorical(output_data[i]).reshape((-1,1))
        targets.append(output_data_ohe)
    items = list(zip(indices, targets))
    
    indices, targets = zip(*items)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)], np.array(targets)

In [0]:
model = load_trained_model_from_checkpoint(
    config_file = paths.config,
    checkpoint_file = paths.checkpoint,
    trainable=True,
    training=True,
    seq_len=SEQ_LEN
)

In [211]:
model.summary()

Model: "model_35"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 391)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 391)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 391, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 391, 768)     1536        Input-Segment[0][0]              
___________________________________________________________________________________________

In [0]:
from keras.layers import Layer

class NonMasking(Layer):   
    def __init__(self, **kwargs):   
        self.supports_masking = True  
        super(NonMasking, self).__init__(**kwargs)   
  
    def build(self, input_shape):   
        input_shape = input_shape   
  
    def compute_mask(self, input, input_mask=None):   
        return None   
  
    def call(self, x, mask=None):   
        return x   
  
    def get_output_shape_for(self, input_shape):   
        return input_shape

In [0]:
import codecs

token_dict = {}
with codecs.open(paths.vocab, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
token_dict_inv = {v: k for k, v in token_dict.items()}
tokenizer = inherit_Tokenizer(token_dict)

In [0]:
from keras_bert import Tokenizer

class inherit_Tokenizer(Tokenizer):
  def _tokenize(self, text):
        if not self._cased:
            text = text
            
            text = text.lower()
        spaced = ''
        for ch in text:
            if self._is_punctuation(ch) or self._is_cjk_character(ch):
                spaced += ' ' + ch + ' '
            elif self._is_space(ch):
                spaced += ' '
            elif ord(ch) == 0 or ord(ch) == 0xfffd or self._is_control(ch):
                continue
            else:
                spaced += ch
        tokens = []
        for word in spaced.strip().split():
            tokens += self._word_piece_tokenize(word)
        return tokens

In [0]:
from keras.layers import merge, dot, concatenate, Flatten
from keras import metrics
def get_bert_finetuning_model(model):
  inputs = model.inputs[:2]
  dense = model.layers[-3].output
  output = keras.layers.Dense(6, activation='softmax',
                              name = 'real_output')(dense)
  bert_model = keras.models.Model(inputs, output)
  bert_model.compile(
      optimizer=RAdam(learning_rate=0.0001, decay=0.0001),
      loss='categorical_crossentropy',
      metrics=['accuracy'])
  
  return bert_model

In [221]:
!pip install keras-radam

In [254]:
from IPython.display import SVG
from keras.utils import model_to_dot
from keras import backend as K
from keras_radam import RAdam

bert_model = get_bert_finetuning_model(model)
bert_model.summary()

Model: "model_40"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 391)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 391)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 391, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 391, 768)     1536        Input-Segment[0][0]              
___________________________________________________________________________________________

In [237]:
train_x, train_y = convert_data(input_data, output_data)

100%|██████████| 2260/2260 [00:00<00:00, 8272.25it/s]


In [257]:
print(np.array(train_x).shape, np.array(input_data).shape, np.array(output_data).shape, np.array(train_y).shape)
print(train_y[0])

(2, 2260, 391) (2260,) (2260,) (2260,)
0.0


In [258]:
sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init = tf.variables_initializer([v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables])
sess.run(init)

history = bert_model.fit(train_x, train_y, epochs=1, batch_size=16, verbose=1, shuffle=True)

ValueError: ignored

In [0]:
bert_model.save_weights("/content/gdrive/My Drive/20200207_bert.h5")